In [1]:
import numpy as np
from stochastic_volatility_models.src.core.underlying import Underlying
from stochastic_volatility_models.src.core.volatility_surface import VolatilitySurface
from stochastic_volatility_models.src.core.pricing_models import PricingModel
from stochastic_volatility_models.src.utils.options.moneyness import moneyness

2024-07-27 16:05:57.862 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [2]:
spx = Underlying("SPX")
expiries = np.array([np.datetime64("2022-03-23"), np.datetime64("2022-03-30"), np.datetime64("2022-04-06")])
strikes = np.array([3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4800, 5000])
spx_vs = VolatilitySurface(
	underlying=spx,
	expiries=expiries,
	strikes=strikes,
	monthly=False,
)
time = np.datetime64("2022-03-09")

In [3]:
prices = spx_vs.empirical_price(time=time)
display(prices.loc[prices["Ask"].isna()])

,Bid,Ask,Mid


In [4]:
from stochastic_volatility_models.visualisations.volatility_surface import plot_volatility_surface

pricing_model = PricingModel(model="Black-Scholes")
fig = plot_volatility_surface(time=time, volatility_surface=spx_vs, quantity_method="empirical_pricing_implied_volatility")
fig.show()

TypeError: VolatilitySurface.empirical_pricing_implied_volatility() missing 1 required positional argument: 'pricing_model'

In [5]:
spx_vs.surface_quantities(time=time, quantity_method="empirical_pricing_implied_volatility", price_types=["Mid"], pricing_model=pricing_model)[0].unstack(0).shape

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 1, 2, 3, 4, 30, 31, 32, 33, 60, 61]

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 1, 2, 30]

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 1, 2, 3, 30, 31, 32, 60]



(3, 15)

In [6]:
spx_vs.empirical_pricing_implied_volatility(time=time, pricing_model=pricing_model)

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 1, 2, 3, 4, 30, 31, 32, 33, 60, 61]

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 1, 2, 30]

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 1, 2, 3, 30, 31, 32, 60]



,Bid,Ask,Mid
SPXW 220323C3400000,0.000000,0.000000,0.000000
SPXW 220323C3500000,0.000000,0.000000,0.000000
SPXW 220323C3600000,0.000000,0.000000,0.000000
SPXW 220323C3700000,0.000000,0.340859,0.000000
SPXW 220323C3800000,0.000000,0.358003,0.303786
...,...,...,...
SPXW 220406P4400000,0.252324,0.261562,0.256949
SPXW 220406P4500000,0.234450,0.248415,0.241489
SPXW 220406P4600000,0.226289,0.249752,0.238367
SPXW 220406P4800000,0.256076,0.298324,0.278983


In [7]:
spx.future_price(time)
moneyness(spx, spx_vs.strikes, time, log=True)

array([ 0.22805665,  0.19906912,  0.17089824,  0.14349926,  0.11683102,
        0.09085553,  0.06553772,  0.04084511,  0.01674756, -0.00678294,
       -0.02977246, -0.05224531, -0.07422422, -0.11678383, -0.15760583])

In [8]:
b_iv = spx_vs.empirical_pricing_implied_volatility(
	pricing_model=PricingModel(model="Black-76"),
	time=time,
)
display(b_iv)

/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [0, 27, 28, 29, 58, 59, 88, 89]



,Bid,Ask,Mid
SPXW 220323C3400000,0.000000,0.612234,0.537993
SPXW 220323C3500000,0.361467,0.557928,0.500292
SPXW 220323C3600000,0.396471,0.511657,0.466751
SPXW 220323C3700000,0.392272,0.466846,0.434175
SPXW 220323C3800000,0.369555,0.431383,0.403092
...,...,...,...
SPXW 220406P4400000,0.239654,0.249093,0.244382
SPXW 220406P4500000,0.215028,0.230123,0.222668
SPXW 220406P4600000,0.191645,0.221560,0.207510
SPXW 220406P4800000,0.000000,0.246686,0.208303
